In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [ ]:
def context(text: str, start: int, end: int, size:int = 20):
    return "... {} ...".format(text[max(0, start - size):min(len(text) - 1, end + size)])
    

In [ ]:
def nerr(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    for span in doc.spans:
        span.normalize(morph_vocab)
    return [{'text': _.text, 'normal': _.normal, 'type': _.type, 
             'context': context(text, _.start, _.stop),
             'start': _.start, 'stop': _.stop
            } for _ in doc.spans]
    

In [ ]:
def date_extract(text):
    dates = list(dates_extractor(text))
    print(dates)
    return [{'text': text[_.start:_.stop], 'match': _, 
             'context': context(text, _.start, _.stop),
             'start': _.start, 'stop': _.stop
            } for _ in dates]

def extract(text: str):
    return {"ner": nerr(text), "date": date_extract(text)}


In [ ]:
text = """Посольство Китая во Франции удалило из своего официального аккаунта в соцсети WeChat расшифровку интервью посла Лу Шая. Стенограмма была доступна несколько часов, но сейчас там, где она была опубликована, стоит пометка «удалена автором».

Лу Шай дал интервью французскому телеканалу LCI 21 апреля. Он, в числе прочего, заявил, что Крым «с самого начала принадлежал России», а Никита Хрущев «подарил Крым Украине во времена Советского Союза». Кроме того, посол Китая во Франции заявил, что страны бывшего СССР «не имеют эффективного статуса в международном праве, потому что не существует международного соглашения, которое придавало бы силу их статусу суверенных стран».

Интервью Лу Шая вызвало огромный резонанс, в частности, его осудили представители стран Балтии и Украины, а МИД КНР пришлось официально заявлять, что Китай уважает суверенитет всех стран, которые ранее входили в состав СССР, а его позиция по Крыму «остается неизменной»."""

extract(text)


In [ ]:
extract("""Президент РФ Владимир Путин подписал указ, предусматривающий «передачу во временное управление» активов юридических и физических лиц из «недружественных стран» в случае изъятия российских активов за рубежом.

Указ, носящий название «О временном управлении некоторым имуществом», Указ переводит под временное управление Росимущества акции немецкой энергетической компании Uniper SE и Fortum Russia B.V. — совместного предприятия с участием финской государственной энергетической компании.

После начала полномасштабного вторжения войск РФ в Украину российские власти ввели ряд ограничений для иностранных компаний и бизнесменов из «недружественных стран», в список которых включены все государства, поддержавшие санкции в отношении России. Так, в частности, продажа компаниями из «недружественных стран» активов в России требует согласования правительственной комиссией. Условием согласования сделок является скидка — не менее 50% от рыночной стоимости активов, рассрочка оплаты или перечисление 10% от суммы сделки в бюджет РФ.""")